<a href="https://colab.research.google.com/github/OverGeek/Face-Recognition-and-Verification/blob/master/Face_Verification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.applications.inception_v3 import InceptionV3
import keras.backend as K
import keras

Using TensorFlow backend.


In [2]:
model = InceptionV3(weights = 'imagenet', include_top = False, input_shape = (96, 96, 3))
x = model.output
#x = keras.layers.AveragePooling2D()(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(128)(x)
predictions = keras.layers.Lambda(lambda  x: K.l2_normalize(x,axis=1))(x)
model = keras.models.Model(inputs=model.input, outputs=predictions)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 96, 96, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 47, 47, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 47, 47, 32)   96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 47, 47, 32)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [3]:
!wget --header="Host: doc-14-9c-docs.googleusercontent.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://drive.google.com/drive/my-drive" --header="Cookie: AUTH_4lgch8kproorp87c5erpsi9rqpbk46dg_nonce=sfd9nfe7nip5s" --header="Connection: keep-alive" "https://doc-14-9c-docs.googleusercontent.com/docs/securesc/0ga3her86at4fpdr2b9n3spfq1b4g359/hpmn37bmlnbtlqlvhcinu4v308u3jirv/1547956800000/14696292948499217652/14696292948499217652/1mINX6s9Ufxqi5_xpCDID8dnRjLbY7zB1?e=download&h=06864318269717761326&nonce=sfd9nfe7nip5s&user=14696292948499217652&hash=t7f80rqpa9s4s9iqkorcg9ndtr0i39lc" -O "lfw-deepfunneled.zip" -c

--2019-01-20 09:36:51--  https://doc-14-9c-docs.googleusercontent.com/docs/securesc/0ga3her86at4fpdr2b9n3spfq1b4g359/hpmn37bmlnbtlqlvhcinu4v308u3jirv/1547956800000/14696292948499217652/14696292948499217652/1mINX6s9Ufxqi5_xpCDID8dnRjLbY7zB1?e=download&h=06864318269717761326&nonce=sfd9nfe7nip5s&user=14696292948499217652&hash=t7f80rqpa9s4s9iqkorcg9ndtr0i39lc
Resolving doc-14-9c-docs.googleusercontent.com (doc-14-9c-docs.googleusercontent.com)... 74.125.124.132, 2607:f8b0:4001:c14::84
Connecting to doc-14-9c-docs.googleusercontent.com (doc-14-9c-docs.googleusercontent.com)|74.125.124.132|:443... connected.
HTTP request sent, awaiting response... 416 Requested range not satisfiable

    The file is already fully retrieved; nothing to do.



In [0]:
!unzip 'lfw-deepfunneled.zip'

In [0]:
cd '/content/'

In [0]:
import os as os
people = sorted(os.listdir('./lfw-deepfunneled'))
print(people)
print('numbe of different identities: ', len(people))

In [0]:
import numpy as np
from PIL import Image
import random
from tqdm import tqdm

anch = []
pos = []
neg = []
for i in tqdm(range(len(people))):
  identity = people[i]
  for j in sorted(os.listdir('lfw-deepfunneled/'+identity+'/'))[:10]:
    anc_image = Image.open('lfw-deepfunneled/'+identity+'/'+j)
    anc_image = anc_image.resize((96, 96))
    for k in sorted(os.listdir('lfw-deepfunneled/'+identity+'/'))[:10]:
      pos_image = Image.open('lfw-deepfunneled/'+identity+'/'+k)
      pos_image = pos_image.resize((96, 96))
      
      neg_id = random.choice(os.listdir('lfw-deepfunneled/'))
      neg_img = random.choice(os.listdir('lfw-deepfunneled/'+neg_id+'/'))
      
      neg_image = Image.open('lfw-deepfunneled/'+neg_id+'/'+neg_img)
      neg_image = neg_image.resize((96, 96))
      
      anch.append(np.array(anc_image))
      pos.append(np.array(pos_image))
      neg.append(np.array(neg_image))

In [0]:
Image.fromarray(anch[5] mode = 'RGB')

In [0]:
Image.fromarray(pos[5], mode = 'RGB')

In [0]:
Image.fromarray(neg[5], mode = 'RGB')

In [0]:
anch = np.array(anch)
anch.shape

In [0]:
pos = np.array(pos)
pos.shape

In [0]:
neg = np.array(neg)
neg.shape

In [0]:
anch_encoding = model.predict(np.expand_dims(train_data[0][0], axis = 0))
pos_encoding = model.predict(np.expand_dims(train_data[1][0], axis = 0))
neg_encoding = model.predict(np.expand_dims(train_data[2][0], axis = 0))

pos_dist = np.sum((anch_encoding - pos_encoding)**2)
neg_dist = np.sum((anch_encoding - neg_encoding)**2)
print(pos_dist)
print(neg_dist)
print(pos_dist - neg_dist+0.2)

In [0]:
train_data = [anch[:39264, :, :, :], pos[:39264, :, :, :], neg[:39264, :, :, :]]

In [0]:
def triplet_loss(y_pred, alpha = 0.2):
    """
    Implementation of the triplet loss as defined by formula (3)
    
    Arguments:
    y_true -- true labels, required when you define a loss in Keras, you don't need it in this function.
    y_pred -- python list containing three objects:
            anchor -- the encodings for the anchor images, of shape (None, 128)
            positive -- the encodings for the positive images, of shape (None, 128)
            negative -- the encodings for the negative images, of shape (None, 128)
    
    Returns:
    loss -- real number, value of the loss
    """
    
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    
    ### START CODE HERE ### (≈ 4 lines)
    # Step 1: Compute the (encoding) distance between the anchor and the positive, you will need to sum over axis=-1
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, positive)), axis=-1)
    # Step 2: Compute the (encoding) distance between the anchor and the negative, you will need to sum over axis=-1
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, negative)), axis=-1)
    # Step 3: subtract the two previous distances and add alpha.
    basic_loss = tf.add(tf.subtract(pos_dist, neg_dist), alpha)
    # Step 4: Take the maximum of basic_loss and 0.0. Sum over the training examples.
    loss = tf.maximum(basic_loss, 0)
    ### END CODE HERE ###

    return loss

In [0]:
with tf.Session() as test:
    tf.set_random_seed(1)
    y_true = (None, None, None)
    y_pred = (tf.random_normal([3, 128], mean=6, stddev=0.1, seed = 1),
              tf.random_normal([3, 128], mean=1, stddev=1, seed = 1),
              tf.random_normal([3, 128], mean=3, stddev=4, seed = 1))
    loss = triplet_loss(y_pred)
    
    print("loss = " + str(loss.eval()))

In [0]:
def identity_loss(y_true, y_pred):
    return K.sum(y_pred)

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.models import Model
import tensorflow as tf

def FRmodel(input_shape):
  
  anch_input = Input(input_shape)
  pos_input = Input(input_shape)
  neg_input = Input(input_shape)
  
  anch = model(anch_input)
  pos = model(pos_input)
  neg = model(neg_input)
  
  res = keras.layers.Lambda(triplet_loss)([anch, pos, neg])
  
  smodel = Model(inputs = [anch_input, pos_input, neg_input], outputs = res)
  return smodel
  

In [0]:
Fmodel = FRmodel((96, 96, 3))

In [0]:
Fmodel.summary()

In [0]:
Fmodel.compile(optimizer = 'adam', loss = identity_loss, metrics = ['mean_absolute_error'])

In [0]:
Fmodel.fit(train_data, np.zeros((train_data[0].shape[0], 1)), epochs = 1, batch_size = 32)

In [0]:
model.save_weights('epochs1.h5')

In [0]:
anch_encoding = model.predict(np.expand_dims(train_data[0][1], axis = 0))
pos_encoding = model.predict(np.expand_dims(train_data[1][1], axis = 0))
neg_encoding = model.predict(np.expand_dims(train_data[2][1], axis = 0))

pos_dist = np.sum((anch_encoding - pos_encoding)**2)
neg_dist = np.sum((anch_encoding - neg_encoding)**2)
print(pos_dist)
print(neg_dist)
print(pos_dist-neg_dist+0.2)

print(anch[:1, :, :, :].shape)
Fmodel.predict([anch[:2, :, :, :], pos[:2, :, :, :], neg[:2, :, :, :]])